In [1]:
import sys
import os

## replace with root project dir
PROJECT_DIR = "/mnt/workspace/__ing/llming/DTC/course"
sys.path.append(PROJECT_DIR)

from utils.huggingface import setup_hf_cache_dir
from utils.utils import load_json_document

from utils.elasticsearch import (
    create_elasticsearch_client,
    search_elasticsearch_indecis,
    load_index_settings,
    create_elasticsearch_index,
    remove_elasticsearch_index,
    index_documents,
)

from utils.query import rag

from transformers import T5Tokenizer, T5ForConditionalGeneration
from openai import OpenAI



setup_hf_cache_dir(os.path.join(PROJECT_DIR, "hf_cache"))

HuggingFace cache directory has been set to: /mnt/workspace/__ing/llming/DTC/course/hf_cache


# Query & Documents

In [2]:
## question
query = 'The course has already started, can I still enroll?'

In [3]:
document_path = f'{PROJECT_DIR}/data/1/documents.json'

documents = load_json_document(document_path)

for _ in documents[10:12]:
    print(_, end="\n\n")

{'text': 'It depends on your background and previous experience with modules. It is expected to require about 5 - 15 hours per week. [source1] [source2]\nYou can also calculate it yourself using this data and then update this answer.', 'section': 'General course-related questions', 'question': 'Course - \u200b\u200bHow many hours per week am I expected to spend on this  course?', 'course': 'data-engineering-zoomcamp'}

{'text': "No, you can only get a certificate if you finish the course with a “live” cohort. We don't award certificates for the self-paced mode. The reason is you need to peer-review capstone(s) after submitting a project. You can only peer-review projects at the time the course is running.", 'section': 'General course-related questions', 'question': 'Certificate - Can I follow the course in a self-paced mode and get a certificate?', 'course': 'data-engineering-zoomcamp'}



# ElasticSearch Client

In [4]:
host = "localhost"
port = 9200

index_name = "course-questions"
index_settings_path=f"{PROJECT_DIR}/config/elasticsearch/course_qa_index_settings.json"
index_settings = load_index_settings(index_settings_path)

recreate_index = False

es_client = create_elasticsearch_client(host, port)
search_elasticsearch_indecis(es_client)

Connected to Elasticsearch


['course-questions']

In [5]:
if recreate_index:
    es_client = create_elasticsearch_client(host, port)
    remove_elasticsearch_index(es_client, index_name)
    create_elasticsearch_index(es_client, index_name, index_settings)


    index_documents(es_client, index_name, documents)

In [6]:
search_elasticsearch_indecis(
        es_client,
)

['course-questions']

# HuggingFace models

## 1. flan-t5-small

In [18]:
model_name = "google/flan-t5-small"

search_context = 'elasticsearch'
boost = {'question': 3.0, 'section': 0.5}
filter_dict={'course': 'data-engineering-zoomcamp'}
num_results = 5
prompt_template_path = os.path.join(PROJECT_DIR,"prompts/course_qa.txt")


rag_params = dict(
    es_client=es_client,
    query=query,
    index_name=index_name,
    filter_dict=filter_dict,
    boost=boost,
    num_results=num_results,
    prompt_template_path=prompt_template_path,
    model_name=model_name,
    search_context=search_context
)

response = rag(**rag_params)
print(response)

Registration is just to gauge interest before the start date.


# Ollama

In [7]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [8]:
model_name = "phi3"

search_context = 'elasticsearch'
boost = {'question': 3.0, 'section': 0.5}
filter_dict={'course': 'data-engineering-zoomcamp'}
num_results = 5
prompt_template_path = os.path.join(PROJECT_DIR,"prompts/course_qa.txt")


rag_params = dict(
    es_client=es_client,
    client=client,
    query=query,
    index_name=index_name,
    filter_dict=filter_dict,
    boost=boost,
    num_results=num_results,
    prompt_template_path=prompt_template_path,
    model_name=model_name,
    search_context=search_context
)

response = rag(**rag_params)
print(response)

 Even if you have already started attending this Data Engineering Bootcamp, I am happy to assist with your enrollment process! Since we're in a self-paced course environment, please feel free to submit questions via Slack or through @ZoomcampQABot for support as needed.

To ensure you are well prepared and can make the most out of this bootcamp experience once it officially begins, I encourage taking some time beforehand by setting up all dependencies such as Google Cloud account, installing Python 3 with Anaconda, Terraform, Git etc. It would also be helpful to review course prerequisites and syllabus just for your reference!

Remember though that you are not required to register prior to starting the bootcamp because registration does serve an additional purpose of gauging interest before its start date - hence there is no specific deadline mentioned in relation with this. It's more important now than ever, as we always suggest avoid procrastinating and try turning your final project